<a href="https://colab.research.google.com/github/robgon-art/InventorBot/blob/main/InventorBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **InventorBot: Using AI to Generate Ideas for Inventions in Any Field**
How a neural network trained on the US Patent Database can produce new and possibly useful inventions.

![alt text](https://raw.githubusercontent.com/robgon-art/InventorBot/main/Shadow_Hand_Bulb_small.png)

# Initialize the InventorBot System
Click the **Run cell** button (the play button) below to set things up. It takes about 5 minutes to load the dataset files and configure the neural network models.

In [ ]:
#@title
%tensorflow_version 1.x
!pip install -q gpt-2-simple
print("Downloading the InventorBot model.")
!gdown --id 1Q5Tvzx2gnTnujP5TGt8yytWvGAzdym5r
!unzip inventor_bot_2.zip

!pip install transformers==3.4.0
from transformers import pipeline
summarizer = pipeline("summarization", model="t5-large", tokenizer="t5-large", framework="pt")
import gpt_2_simple as gpt2
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)
import textwrap
wrapper = textwrap.TextWrapper(width=175) 
import os
import sys
old_stderr = sys.stderr
f = open(os.devnull, 'w')
sys.stderr = f
setup_gruen = False

# Create Some New Ideas for Inventions
Type in some **keywords** then click **Run cell** button to generate new invention ideas with titles and abstracts that relate to the keywords. You can run this multiple times with the same or different keywords. You can specify the **temperature** to change the quality and diversity of the text. Higher temperatures will have greater linguistic variety where lower temperatures will be more grammatically correct. You can also specify the **number of ideas** to generate.

In [10]:
keywords = "improved headphones" #@param {type:"string"}
temperature = 85 #@param {type:"slider", min:70, max:100, step:1}
number_of_ideas = 7 #@param {type:"slider", min:4, max:20, step:1}

new_ideas = gpt2.generate(sess, length=200, temperature=temperature/100,
  prefix="KEYWORDS: " + keywords + "\nTITLE:", nsamples=number_of_ideas, 
  batch_size=number_of_ideas, return_as_list=True, include_prefix=True,
  truncate="***")

keepers = []
summaries = []

show_full_abstract = False

for idea in new_ideas:
  if idea.count("KEYWORDS:") is 1 and idea.count("TITLE:") is 1 and idea.count("ABSTRACT:") is 1:
    parts = idea.split("\n")
    if len(parts) is not 4:
      continue
    keeper = []
    for part in parts:
      if part.startswith("ABSTRACT:"):
        abstract = part[9:]
        if show_full_abstract:
          part = "FULL ABSTRACT: " + abstract + "\n"
        else:
          part = ""
        summary_text = summarizer(abstract, max_length=200, min_length=5, do_sample=False)[0]['summary_text']
        summary_text = summary_text + " "
        summary_text ='. '.join([i.capitalize() for i in summary_text.split(' . ')])
        summaries.append(summary_text)
        part += "ABSTRACT: " + summary_text
      if len(part) > 0:
        keeper.append(part)
      print(wrapper.fill(part))
    keepers.append(keeper)

KEYWORDS: improved headphones
TITLE: Headphone system including improved headphones
ABSTRACT: Headphone system includes an audio system and a method for optimizing speaker efficiency. Headphone systems and microphones may be in different modes or in different
applications. Sound source selection may be accomplished by sensing the direction of the audio signals on the network or within the headphones.

KEYWORDS: improved headphones
TITLE: Audio amplifier with improved headphones
ABSTRACT: Sound amplifiers are formed using primary and secondary analog analog amplification stages. Amplifier stages have a base region having a signal collector region.

KEYWORDS: improved headphones
TITLE: Method for producing improved headphones
ABSTRACT: The present invention provides a method for producing improved headphones. The method includes laying a pair of headphones on a front surface of a substrate.

KEYWORDS: improved headphones
TITLE: Radio frequency signals for improved headphones
ABSTRACT: Fi